In [3]:
class MyClass:
    x = 5


In [5]:
p1 = MyClass()
print(p1.x)

5


In [15]:
class test:
    def __init__(self, num):
        self.x = num
    def add(self, num):
        self.x += num
    def sub(self, num):
        self.x -= num
    def mul(self, num):
        self.x *= num
    def div(self, num):
        self.x /= num
    def get(a,b):
        return a.x

c = test(5)

c.add(2)

c.get(2)


7

In [17]:
import numpy as np
from sklearn.model_selection import train_test_split


In [8]:

class modelManager:
    """
    This is a class to manage the different models, to have the same train test split when using multiple models
    Takes the full dataframe, X and target y as input
    """
    def __init__(self, data, X, y) -> None:
        self.data = data
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        

9